In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

from datamodule import PromptDataset, get_length
from model_baseline import DistilBertClassifier
import tiktoken



In [5]:
import pandas as pd

# === Tokenizer ===
tokenizer_BERT = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_GPT2 = AutoTokenizer.from_pretrained("gpt2")

steps = [32, 64, 128, 256, 512]

# === Load data ===
df_train = pd.read_csv(r"..\data\train.csv")
train_data = list(zip(df_train["user_prompt"], get_length(df_train["model_response"], tokenizer_GPT2, max_length=512, steps= steps)))
del df_train

df_test = pd.read_csv(r"..\data\test.csv")
df_test = df_test.dropna(subset=["model_response"])
data_test = list(zip(df_test["user_prompt"], get_length(df_test["model_response"], tokenizer_GPT2, max_length= 512, steps= steps)))
del df_test

val_data, test_data = train_test_split(data_test, test_size=0.3, random_state=42)

# All the training prompt except one have length < 64
train_ds = PromptDataset(train_data, tokenizer_BERT, max_len=64)
val_ds = PromptDataset(val_data, tokenizer_BERT, max_len=128)
test_ds = PromptDataset(test_data, tokenizer_BERT, max_len=128)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32)
test_dl = DataLoader(test_ds, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


c:\Users\lgand\anaconda3\envs\DL\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
model = DistilBertClassifier(n_classes=5)
model = model.to(device)

In [ ]:
@torch.no_grad()
def estimate_loss(eval_iters = 10):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        if split == 'train':
            dataloader = train_dl
        else:
            dataloader = val_dl
        k = 0
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits, loss = model(input_ids, attention_mask, labels)
            assert loss is not None, "Loss should not be None"
            losses[k] = loss.item()
            k += 1
            if k >= eval_iters:
                break
        out[split] = losses.mean()
    model.train()
    return out

### Freezing the DistilBERT parameters

In [6]:
for param in model.encoder.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=2e-5)

In [ ]:
eval_interval = 200
max_iters = len(train_dl)

# === Training loop ===
for epoch in range(10):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_dl):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits, loss = model(input_ids, attention_mask, labels)
        assert loss is not None, "Loss should not be None"
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if i % eval_interval == 0 or i == max_iters - 1:
                losses = estimate_loss()
                print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

In [180]:
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=1e-5)


In [ ]:
eval_interval = 200
max_iters = len(train_dl)

# === Training loop ===
for epoch in range(15):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_dl):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits, loss = model(input_ids, attention_mask, labels)
        assert loss is not None, "Loss should not be None"
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if i % eval_interval == 0 or i == max_iters - 1:
                losses = estimate_loss()
                print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

### Fine tuning: changing also the parameters of DistilBERT

In [ ]:
# Congela tutti i parametri di BERT
for param in model.encoder.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-6)

In [ ]:
eval_interval = 200
max_iters = len(train_dl)

# === Training loop ===
for epoch in range(8):
    model.train()
    total_loss = 0
    for i, batch in enumerate(train_dl):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        logits, loss = model(input_ids, attention_mask, labels)
        assert loss is not None, "Loss should not be None"
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if i % eval_interval == 0 or i == max_iters - 1:
                losses = estimate_loss()
                print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    print(f"[Epoch {epoch+1}] Loss: {total_loss:.4f}")

In [ ]:
torch.save(model.state_dict(), "checkpoints/DistilBERT_1.pth")


### Evaluation

In [ ]:
model = DistilBertClassifier(n_classes=5)
model.load_state_dict(torch.load("checkpoints/DistilBERT_1.pth"))
model = model.to(device)
model.eval()
0

0

In [ ]:
input_text = "Can you explain the theory of relativity?"
input_enc = tokenizer_BERT(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).to(device)
output = model(input_enc['input_ids'], input_enc['attention_mask'])
print(output)

(tensor([[-3.0026, -1.5511,  0.2183,  1.3869,  1.6006]], device='cuda:0',
       grad_fn=<AddmmBackward0>), None)


In [ ]:
input_text = "What's your name?"
input_enc = tokenizer_BERT(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).to(device)
output = model(input_enc['input_ids'], input_enc['attention_mask'])
print(output)

(tensor([[ 3.0997,  2.6984,  1.7371, -2.1407, -5.9127]], device='cuda:0',
       grad_fn=<AddmmBackward0>), None)


In [ ]:
input_text = "What is 3 + 3?"
input_enc = tokenizer_BERT(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).to(device)
output = model(input_enc['input_ids'], input_enc['attention_mask'])
print(output)

(tensor([[ 1.2852,  0.6386,  1.1362, -0.5773, -2.8144]], device='cuda:0',
       grad_fn=<AddmmBackward0>), None)


In [23]:
@torch.no_grad()
def see_prediction(kappa = 1):
    out = {}
    model.eval()
    for split in ['val']:
        losses = torch.zeros(kappa)
        if split == 'train':
            dataloader = train_dl
        else:
            dataloader = val_dl
        k = 0
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits, loss = model(input_ids, attention_mask, labels)
            pred = torch.argmax(logits, dim=1)
            print(f"pred: {[(pred[i].item(), labels[i].item()) for i in range(32)]}")
            assert loss is not None, "Loss should not be None"
            losses[k] = loss.item()
            k += 1
            if k >= kappa:
                break
        out[split] = losses
    model.train()
    print("mean cross entropy loss: ", out["val"].mean())
    return 

In [24]:
see_prediction(10)

pred: [(2, 1), (2, 2), (4, 3), (2, 3), (1, 0), (3, 3), (3, 3), (3, 4), (4, 4), (2, 2), (2, 2), (3, 3), (4, 4), (4, 4), (0, 0), (2, 4), (3, 4), (3, 3), (2, 2), (4, 3), (3, 2), (4, 2), (4, 4), (2, 3), (3, 4), (1, 2), (4, 3), (4, 4), (4, 4), (3, 3), (3, 3), (2, 2)]
pred: [(2, 2), (4, 3), (3, 2), (2, 0), (4, 4), (4, 4), (3, 3), (3, 2), (2, 4), (4, 4), (4, 3), (2, 2), (4, 3), (3, 0), (4, 4), (3, 3), (3, 3), (3, 4), (4, 3), (2, 2), (3, 3), (3, 2), (3, 3), (1, 1), (4, 4), (3, 2), (1, 0), (3, 4), (4, 4), (2, 1), (3, 3), (3, 0)]
pred: [(3, 4), (3, 3), (4, 4), (4, 2), (3, 2), (2, 2), (2, 2), (3, 3), (0, 0), (1, 1), (4, 4), (4, 4), (4, 3), (4, 4), (3, 1), (3, 3), (4, 3), (3, 2), (4, 4), (3, 3), (3, 2), (4, 4), (3, 4), (3, 3), (2, 3), (3, 3), (4, 2), (4, 3), (3, 2), (2, 2), (3, 4), (4, 4)]
pred: [(3, 4), (2, 0), (4, 4), (2, 0), (2, 2), (3, 4), (4, 3), (4, 4), (4, 4), (4, 3), (2, 3), (0, 0), (3, 2), (3, 4), (2, 2), (0, 1), (3, 3), (2, 2), (3, 2), (2, 2), (0, 0), (3, 3), (2, 2), (4, 3), (4, 4), (3, 

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error


def evaluate_accuracy(model, dataloader, device='cpu'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits, loss = model(input_ids, attention_mask, labels)
            pred = torch.argmax(logits, dim=1)

            all_preds.extend(pred.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(all_labels, all_preds)
    mse = mean_squared_error(all_labels, all_preds)
    return accuracy, mse



In [33]:
acc, mse = evaluate_accuracy(model, val_dl, device=device)
print(f"Accuracy: {acc:.2%}")
print(f"Mean Squared Error: {mse:.4f}")


Accuracy: 48.67%
Mean Squared Error: 1.0368
